In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

J:\\Projects\\JHUTrabeculae\\Results\\TestSofware\\CenterOfSegmentationProblem

In [3]:
import os
# Python packages
PythonTools_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\PythonTools'
pyimagej_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\pyimagej'
TrabeculaeTools_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\TrabeculaeTools'
elastix_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Programs\\elastix'
ImageJ_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Programs\\Fiji.app'
Macros_DIR = 'J:\\Projects\\JHUTrabeculae\\Software\\Python\\TrabeculaeTools\\BoneJMacros'

# Elastix registration
elastix_EXE = os.path.join(elastix_DIR,'elastix.exe')

# Path to imageJ (Need BoneJ and NII reader plugins)
ImageJ_EXE = os.path.join(ImageJ_DIR,'ImageJ-win64.exe')

# Macro definition for segmentation Trabeculae
SegmentfileDescription_XML = os.path.join(Macros_DIR,'SegmentTrabeculaImageJMacro.xml')
Segmentmacro_IJM = os.path.join(Macros_DIR,'SegmentTrabeculaImageJMacro.ijm')

# Macro definition for Metric Trabeculae
MetricfileDescription_XML= os.path.join(Macros_DIR,'BoneJMetricsImageJMacro.xml')
Metricmacro_IJM = os.path.join(Macros_DIR,'BoneJMetricsImageJMacro.ijm')

In [5]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
import SimpleITK as sitk
from scipy.spatial.distance import dice, jaccard, matching, rogerstanimoto, russellrao, sokalmichener, sokalsneath, yule

# Add Python Tools to path
sys.path.append(PythonTools_DIR)

# Add pyimagej
sys.path.append(pyimagej_DIR)

# Add TrabeculaeTools
sys.path.append(TrabeculaeTools_DIR)

In [7]:
# PythonTools imports
from PythonTools.helpers.elastix import elastix as elastixhelper
from PythonTools import io, transformations

# pyimagej imports
from pyimagej.pyimagej import ImageJ, MacroImageJ

# TrabeculaeTools imports
from TrabeculaeTools.analysis import DownsampleImage, CreateRoIfileStudy, CreateRoI, CreateRoITransformed, PrintPercentage, ResampleImage
from TrabeculaeTools.analysis import SegmentTrabeculaeBoneJ, GetResultsFromSegmentation, GetResultsForComparison, LocalThresholding, SegmentTrabeculaeLocalThresholding

In [9]:
CBCTimagePath = 'J:/Projects/JHUTrabeculae/Results/TestSofware/CenterOfSegmentationProblem/CBCT_roi.nii' 
uCTimagePath = 'J:/Projects/JHUTrabeculae/Results/TestSofware/CenterOfSegmentationProblem/uCT_roi.nii' 
uCTimagePath_segmentation = 'J:/Projects/JHUTrabeculae/Results/TestSofware/CenterOfSegmentationProblem/uCT_roi_segmentedBoneJ.nii' 

uCTResampledimagePath = 'J:/Projects/JHUTrabeculae/Results/TestSofware/CenterOfSegmentationProblem/uCT_roi_resampled.nii' 
uCTResampledimagePath_segmentation = 'J:/Projects/JHUTrabeculae/Results/TestSofware/CenterOfSegmentationProblem/uCT_roi_resampled_segmentedBoneJ.nii'

dataFile = 'J:/Projects/JHUTrabeculae/Results/TestSofware/CenterOfSegmentationProblem/ResultsAllMetrics.csv'

In [10]:
## Create uCT image with CBCT voxel size
uCTimageCBCTVoxelSize = ResampleImage(CBCTimagePath,uCTimagePath)
sitk.WriteImage(uCTimageCBCTVoxelSize,uCTResampledimagePath)

In [12]:
uCTdataSegmentation = SegmentTrabeculaeBoneJ(   ImageJ_EXE, Segmentmacro_IJM, SegmentfileDescription_XML, 
                                                    defaultTimeout = 700,\
                                                    PathToRoIfile = uCTimagePath,\
                                                    PathToSegmentedRoIfile = uCTimagePath_segmentation,\
                                                    SMOOTH_Sigma = 0.03,\
                                                    TH_Erosion = 0,\
                                                    TH_Dilation = 0)

In [15]:
original = sitk.Cast(sitk.ReadImage(uCTimagePath), sitk.sitkFloat32)
segmented = sitk.Cast(sitk.ReadImage(uCTimagePath_segmentation), sitk.sitkFloat32)


print original.GetOrigin(), segmented.GetOrigin()
segmented.SetOrigin(original.GetOrigin())
print original.GetOrigin(), segmented.GetOrigin()

(7.800000190734863, 8.25, 3.5250000953674316) (0.0, 0.0, 0.0)
(7.800000190734863, 8.25, 3.5250000953674316) (7.800000190734863, 8.25, 3.5250000953674316)
